In [2]:
from boson_data_lib import *
import numpy as np
import scipy.linalg as sl
import os
import pandas as pd
import glob

In [3]:
data_dir = "../DATA/"
tests_dir = "../TESTS/"

In [4]:
best_init_rho =  np.array([[ 0.96693975+0.j        , -0.17832301+0.00560344j],
                               [-0.17832301-0.00560344j,  0.03306025+0.j        ]])

fid_dists = fidelity_distances(data_dir, best_init_rho);

In [12]:
def read_as_dataframe(filename, fid_distances, data_dir, tests_dir):

    df = pd.DataFrame()
    
    print("Processing...")
    
    with h5py.File(tests_dir+filename, "r") as f:
        
        #gammas = ['0.079477', '0.25133', '0.79477', '2.5133', '7.9477', '25.133', '79.477', '251.33']
        gammas = f.keys()
        
        for gamma in gammas:
            
            print(gamma)
            
            g = gamma[6:]
            
            init_states = f[gamma].keys()
            
            for state in init_states:
                
                F_lme_exact = f[gamma][state]["F_lme_exact"][()]
              
                
                ser_len = len(F_lme_exact)

                gamma_column = [g] * ser_len
                state_column = [state[7:]] * ser_len

                fid_dist_column = [fid_distances[int(state[7:])-1]] * ser_len      

                f_name = data_dir + "/" + state + "_2CUT_data.h5"
                #f_name = data_dir + "\\" + state + "_2CUT_data.h5"
                t, dt = extract_time(f_name, g)
                time_column = t
                gamma_time_column = np.array(t)*float(g) 

                d_ser = {'Gamma': gamma_column,
                         'State': state_column,
                         'Time': time_column, 
                         'gt': gamma_time_column,
                         'F_lme_exact': F_lme_exact,
                         'Infidelity': 1-F_lme_exact,
                         'Distance': fid_dist_column}

                df_ser = pd.DataFrame(data = d_ser)   
                df = pd.concat([df, df_ser])
    
    print(" done!")
    
    return df

In [13]:
test_file = "DMD_TF_POP_random_trn4_tst20_2024-Jan-13_at_17-45.h5"

In [14]:
df = read_as_dataframe(test_file, fid_dists, data_dir, tests_dir)

Processing...
gamma_0.079477
gamma_0.25133
gamma_0.79477
gamma_2.5133
gamma_25.133
gamma_251.33
gamma_7.9477
gamma_79.477
 done!


In [15]:
df

,Gamma,State,Time,gt,F_lme_exact,Infidelity,Distance
0,0.079477,1,0.06000,0.004769,1.000000,-6.039613e-14,0.341925
1,0.079477,1,0.08000,0.006358,1.000000,4.559227e-07,0.341925
2,0.079477,1,0.10000,0.007948,0.999994,5.502275e-06,0.341925
3,0.079477,1,0.12000,0.009537,0.999993,7.269960e-06,0.341925
4,0.079477,1,0.14000,0.011127,0.999999,8.530532e-07,0.341925
...,...,...,...,...,...,...,...
751,79.477,9,0.23800,18.915526,0.993241,6.759291e-03,0.277180
752,79.477,9,0.23825,18.935395,0.993225,6.775136e-03,0.277180
753,79.477,9,0.23850,18.955264,0.993209,6.790614e-03,0.277180
754,79.477,9,0.23875,18.975134,0.993194,6.805716e-03,0.277180


In [16]:
pkl_name = "dataframe_"+test_file[:-3]+".pkl"

df.to_pickle(tests_dir+pkl_name)

In [17]:
pkl_name

'dataframe_DMD_TF_POP_random_trn4_tst20_2024-Jan-13_at_17-45.pkl'

In [18]:
tests_dir+pkl_name

'../TESTS/dataframe_DMD_TF_POP_random_trn4_tst20_2024-Jan-13_at_17-45.pkl'